##Dublin Business School
##Module: B9BA102_Applied Statistics & Machine Learning
##CA TWO - Group Assignment
##Laptop price prediction by:
      1. Manisha        -  20024385
      2. Abhishek Sen   -  20012367
      3. Kashmira Ghag  -  20023193


##Data Pre-processing

In [2]:
pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 2.0 MB/s eta 0:00:00


In [3]:
# Module import from Python libraries
from pandas import read_csv,Series,DataFrame,get_dummies
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDRegressor
from category_encoders import TargetEncoder
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from plotly import graph_objs, figure_factory
from sklearn import linear_model

# Data reading

data1 = read_csv("/content/drive/MyDrive/Colab/Laptop Price Prediction.csv")
data1.head()
data1.shape
data1.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522 entries, 0 to 1521
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1522 non-null   object 
 1   TypeName     1522 non-null   object 
 2   Screen_size  1522 non-null   object 
 3   IPS_panel    1522 non-null   int64  
 4   PPI          1522 non-null   int64  
 5   Touchscreen  1522 non-null   int64  
 6   CPU_brand    1522 non-null   object 
 7   Ram          1522 non-null   int64  
 8   Memory_type  1521 non-null   object 
 9   GPU_brand    1522 non-null   object 
 10  OpSys        1522 non-null   object 
 11  Weight       1522 non-null   float64
 12  Price        1522 non-null   int64  
dtypes: float64(1), int64(5), object(7)
memory usage: 154.7+ KB


In [5]:
DataFrame(data1['Screen_size'])
print(data1['Screen_size'].unique())
DataFrame(data1['CPU_brand'])
print(data1['CPU_brand'].unique())

['small' 'large' 'medium']
['Intel' 'AMD' 'Samsung']


In [6]:
# Encoding using .map for binary features
data1['Screen_size'] = data1['Screen_size'].map({'small':2,'medium':1,'large':0})
data1['CPU_brand'] = data1['CPU_brand'].map({'Intel':2,'AMD':1,'Samsung':0})


In [7]:
# Data splitting
X_F = data1.drop(['Price'], axis = 1) # Features
Y = data1['Price'] # Label
print(X_F.shape)
print(Y.shape)


(1522, 12)
(1522,)


In [8]:
# Feature with their attributes and count
print(data1['Company'].value_counts())
print(data1['TypeName'].value_counts())
print(data1['Memory_type'].value_counts())
print(data1['GPU_brand'].value_counts())
print(data1['OpSys'].value_counts())

Company
Lenovo       345
Dell         328
HP           317
Asus         186
Acer         132
MSI           66
Toshiba       58
Apple         27
Mediacom      10
Samsung        9
Razer          7
Microsoft      6
Google         5
Xiaomi         5
Vero           5
LG             5
Chuwi          4
Fujitsu        4
Huawei         3
Name: count, dtype: int64
TypeName
Notebook              855
Gaming                248
Ultrabook             228
2 in 1 Convertible    131
Workstation            34
Netbook                26
Name: count, dtype: int64
Memory_type
SSD              753
HDD              425
Hybrid           261
Flash Storage     82
Name: count, dtype: int64
GPU_brand
Intel     833
Nvidia    478
AMD       210
ARM         1
Name: count, dtype: int64
OpSys
windows    1316
other       107
linux        72
macos        27
Name: count, dtype: int64


In [9]:
#Target Encoding
Target_Encode = TargetEncoder(cols=['Company','TypeName','Memory_type','GPU_brand','OpSys']).fit(X_F,Y)
X = Target_Encode.transform(X_F)
print(X.info())
X.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1522 entries, 0 to 1521
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Company      1522 non-null   float64
 1   TypeName     1522 non-null   float64
 2   Screen_size  1522 non-null   int64  
 3   IPS_panel    1522 non-null   int64  
 4   PPI          1522 non-null   int64  
 5   Touchscreen  1522 non-null   int64  
 6   CPU_brand    1522 non-null   int64  
 7   Ram          1522 non-null   int64  
 8   Memory_type  1522 non-null   float64
 9   GPU_brand    1522 non-null   float64
 10  OpSys        1522 non-null   float64
 11  Weight       1522 non-null   float64
dtypes: float64(6), int64(6)
memory usage: 142.8 KB
None


,Company,TypeName,Screen_size,IPS_panel,PPI,Touchscreen,CPU_brand,Ram,Memory_type,GPU_brand,OpSys,Weight
0,58726.242902,84540.754364,2,1,352,1,2,8,70064.969456,54190.200480,63590.332067,0.97
1,59529.682844,41543.043275,0,0,127,0,2,8,83386.785440,78711.441423,63590.332067,2.10
2,76735.634625,84540.754364,2,1,227,0,2,8,70064.969456,54190.200480,76735.634625,1.37
3,59529.682844,41543.043275,0,0,91,0,2,4,34902.541176,78711.441423,63590.332067,2.80
4,59529.682844,41543.043275,1,0,141,0,2,4,34902.541176,54190.200480,63590.332067,2.00


In [10]:
# Heatmap for feature reduction
correlation  = X.corr()
# print (correlation)
Heatmap = figure_factory.create_annotated_heatmap(correlation.values,list(correlation.columns),list(correlation.columns),correlation.round(2).values,showscale=True)
Heatmap.show()
# No feature found to be drops in heatmap.


In [11]:
# Data Scaling/Transformation
X_ = StandardScaler().fit_transform(X)
DataFrame(X_)

,0,1,2,3,4,5,6,7,8,9,10,11
0,-0.102985,1.068529,1.637328,1.570659,4.665942,2.513425,0.232491,-0.089666,0.505957,-0.472607,0.333888,-1.370908
1,-0.026709,-0.817542,-1.855333,-0.636675,-0.433574,-0.397863,0.232491,-0.089666,1.197514,1.414606,0.333888,0.020353
2,1.606774,1.068529,1.637328,1.570659,1.832877,-0.397863,0.232491,-0.089666,0.505957,-0.472607,1.620716,-0.878426
3,-0.026709,-0.817542,-1.855333,-0.636675,-1.249497,-0.397863,0.232491,-0.830351,-1.319382,1.414606,0.333888,0.882196
4,-0.026709,-0.817542,-0.109002,-0.636675,-0.116271,-0.397863,0.232491,-0.830351,-1.319382,-0.472607,0.333888,-0.102768
...,...,...,...,...,...,...,...,...,...,...,...,...
1517,-0.026709,-0.817542,-0.109002,-0.636675,0.246361,-0.397863,0.232491,-0.089666,0.505957,-0.472607,0.333888,-0.102768
1518,2.854832,1.397242,-1.855333,-0.636675,-0.433574,-0.397863,0.232491,1.391706,1.197514,1.414606,0.333888,2.975243
1519,0.404262,-0.817542,-0.109002,-0.636675,-1.045516,-0.397863,0.232491,-0.830351,0.505957,-0.472607,0.333888,0.118849
1520,-0.185598,-0.817542,-0.109002,-0.636675,-0.116271,-0.397863,0.232491,-0.830351,-1.319382,-0.472607,0.333888,-0.225888


##Building Regressor - Linear Regression (4 Methods)

In [30]:
# Linear Regression (LR) with 'None' penalty

LinearRegression1 = SGDRegressor(random_state = 1, penalty = None) # Building Regressor
Hparameter1 = {'eta0': [.00001,.0001, .001, .01,1], 'max_iter':[500,800,1100,1400,1700]}
grid_search1 = GridSearchCV(estimator=LinearRegression1, param_grid=Hparameter1, scoring='r2', cv=5)
grid_search1.fit(X_, Y) #Traning, Testing, Evaluation, Ranking

best_parameters = grid_search1.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search1.best_score_
print("Best result: ", best_result)
best_model = grid_search1.best_estimator_
print("Intercept β0: ", best_model.intercept_)
print(DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

# Modified mean square error
r,c=X_.shape
modified_r2 = 1-(1-best_result)*((4/5-1)/5*r-1)/((4/5-1)/5*r-c-1) # 4/5*r is number of rows in training set, c is number of columns
print("modified_r2: ", modified_r2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number o

Best parameters:  {'eta0': 0.0001, 'max_iter': 1100}
Best result:  0.6657131589629689
Intercept β0:  [60194.87537302]
       Features  Coefficients
7           Ram  13376.024323
4           PPI   9854.864980
1      TypeName   9843.414013
8   Memory_type   4557.487864
10        OpSys   3671.878937
6     CPU_brand   3031.291983
0       Company   2117.679856
5   Touchscreen   1096.129267
11       Weight    968.017295
3     IPS_panel    228.444088
9     GPU_brand   -367.418888
2   Screen_size  -5308.771390
modified_r2:  0.7200098846322214


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.



In [31]:
# Linear Regression (LR) using 'L1' Regularization

LinearRegression2 = linear_model.SGDRegressor(random_state = 1, penalty = 'l1') # Building Regressor
Hparameter2 = {'eta0': [.00001,.0001, .001, .01,1], 'max_iter':[500,800,1100,1400,1700],'alpha': [.0001,.001, .01, .1, 1]}
grid_search2 = GridSearchCV(estimator=LinearRegression2, param_grid=Hparameter2, scoring='r2', cv=5)
grid_search2.fit(X_,Y) #Traning, Testing, Evaluation, Ranking

best_parameters = grid_search2.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search2.best_score_
print("Best result: ", best_result)
best_model = grid_search2.best_estimator_
print("Intercept β0: ", best_model.intercept_)
print(DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

#Modified mean square error
r,c=X_.shape
modified_r2 = 1-(1-best_result)*((4/5-1)/5*r-1)/((4/5-1)/5*r-c-1)
print("modified_r2: ", modified_r2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number o

Best parameters:  {'alpha': 1, 'eta0': 0.0001, 'max_iter': 1100}
Best result:  0.6657236491286509
Intercept β0:  [60194.87527688]
       Features  Coefficients
7           Ram  13375.818116
4           PPI   9853.861768
1      TypeName   9842.294001
8   Memory_type   4556.881253
10        OpSys   3671.317621
6     CPU_brand   3029.022590
0       Company   2117.024456
5   Touchscreen   1095.374888
11       Weight    966.200974
3     IPS_panel    227.397466
9     GPU_brand   -362.137113
2   Screen_size  -5306.221243
modified_r2:  0.720018670926921


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.



In [32]:
# Linear Regression (LR) using 'L2' Regularization
LinearRegression3 = linear_model.SGDRegressor(random_state = 1, penalty = 'l2') # Building Regressor
Hparameter3 = {'eta0': [.00001,.0001, .001, .01,1], 'max_iter':[500,800,1100,1400,1700],'alpha': [.0001,.001, .01, .1, 1]}
grid_search3 = GridSearchCV(estimator=LinearRegression3, param_grid=Hparameter3, scoring='r2', cv=5)
grid_search3.fit(X_,Y) #Traning, Testing, Evaluation, Ranking

best_parameters = grid_search3.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search3.best_score_
print("Best result: ", best_result)
best_model = grid_search3.best_estimator_
print("Intercept β0: ", best_model.intercept_)
print(DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

# Modified mean square error
r,c=X_.shape
modified_r2 = 1-(1-best_result)*((4/5-1)/5*r-1)/((4/5-1)/5*r-c-1)
print("modified_r2: ", modified_r2)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number o

Best parameters:  {'alpha': 0.01, 'eta0': 0.0001, 'max_iter': 1100}
Best result:  0.6658036957763722
Intercept β0:  [60194.87638408]
       Features  Coefficients
7           Ram  13269.142644
1      TypeName   9769.749044
4           PPI   9754.739898
8   Memory_type   4578.994713
10        OpSys   3657.383325
6     CPU_brand   2994.575083
0       Company   2156.248052
5   Touchscreen   1123.503542
11       Weight    997.036075
3     IPS_panel    273.244489
9     GPU_brand   -285.009604
2   Screen_size  -5208.741556
modified_r2:  0.7200857159534639


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.



In [33]:
# Linear Regression using 'Elasticnet' Regularization

LinearRegression4 = SGDRegressor(random_state = 1, penalty = 'elasticnet')
Hparameter4 = {'eta0': [.0001,.001, .01, .1, 1], 'max_iter':[50,100,200,500,800],'alpha': [.0001,.001, .01, .1, 1], 'l1_ratio': [0,0.25,0.5,0.75,1]}
grid_search4 = GridSearchCV(estimator=LinearRegression4, param_grid=Hparameter4, scoring='r2', cv=5)
grid_search4.fit(X_, Y) #Traning, Testing, Evaluation, Ranking

best_parameters = grid_search4.best_params_
print("Best parameters: ", best_parameters)
best_result4 = grid_search4.best_score_
print("Best result: ", best_result)
best_model = grid_search4.best_estimator_
print("Intercept β0: ", best_model.intercept_)
print(DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

# Modified mean square error
r, c=X_.shape
modified_r2 = 1-(1-best_result)*((4/5-1)/5*r-1)/((4/5-1)/5*r-c-1)
print("modified_r2: ", modified_r2)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number o

Best parameters:  {'alpha': 0.1, 'eta0': 0.001, 'l1_ratio': 0.75, 'max_iter': 50}
Best result:  0.6658036957763722
Intercept β0:  [60176.42559944]
       Features  Coefficients
7           Ram  13107.392951
1      TypeName   9644.147824
4           PPI   9569.840110
8   Memory_type   4616.235568
10        OpSys   3632.954011
6     CPU_brand   2936.339100
0       Company   2222.257270
5   Touchscreen   1166.009795
11       Weight   1061.448064
3     IPS_panel    342.159094
9     GPU_brand   -144.039686
2   Screen_size  -5026.197949
modified_r2:  0.7200857159534639


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_stochastic_gradient.py:1548: ConvergenceWarning:

Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.



##Ramdon Forest Regressor

In [12]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
cv=5
RF_Regressor1 = RandomForestRegressor(criterion='squared_error', max_features='sqrt', random_state=1)
no_Trees = {'n_estimators': [350,400,450,500,550,600]}
grid_search3 = GridSearchCV(estimator=RF_Regressor1, param_grid=no_Trees, scoring='r2')
grid_search3.fit(X_, Y)

best_parameters = grid_search3.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search3.best_score_
print("best_score: ", best_result)
r, c=X_.shape
modified_r2 = 1-(1-best_result)*((4/5-1)/5*r-1)/((4/5-1)/5*r-c-1)
print("modified_r2: ", modified_r2)
Important_feature = Series(grid_search3.best_estimator_.feature_importances_, index=list(X)).sort_values(ascending=False) # Getting feature importances list for the best model
print(Important_feature)
best_model = grid_search3.best_estimator_

Best parameters:  {'n_estimators': 450}
best_score:  0.8482672289033225
modified_r2:  0.8729125084534055
Ram            0.266994
TypeName       0.196589
Weight         0.163931
PPI            0.115661
Memory_type    0.091619
Company        0.054952
Screen_size    0.029578
GPU_brand      0.026675
IPS_panel      0.020960
OpSys          0.014754
Touchscreen    0.014710
CPU_brand      0.003579
dtype: float64


##Support Vector Regression (2 Methods)

In [35]:
#Support Vector Regression without using C Hyperparameter

SVRegressor = SVR() #Random_state=1 was not accepted
cv=8
Hparameters = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'epsilon': [100,500,1000,1500]}
grid_search6 = GridSearchCV(estimator=SVRegressor, param_grid=Hparameters, scoring='r2')
grid_search6.fit(X_, Y) #Traning, Testing, Evaluation, Ranking
best_parameters = grid_search6.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search6.best_score_
print("Best result: ", best_result)

# Modified mean square error
r,c=X_.shape
modified_r2 = 1-(1-best_result)*((cv-1)/cv*r-1)/((cv-1)/cv*r-c-1)
print("modified_r2: ", modified_r2)

Best parameters:  {'epsilon': 500, 'kernel': 'linear'}
Best result:  0.03600840798570184
modified_r2:  0.027236541366424838


In [36]:
# Support Vector Regression using C Hyperparameter

SVRegressor = SVR() #Random_state=1 was not accepted
cv=8
Hparameters = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [1000000,2000000,3000000,4000000], 'epsilon': [1000,10000,20000]}
grid_search7 = GridSearchCV(estimator=SVRegressor, param_grid=Hparameters, scoring='r2')
grid_search7.fit(X_, Y) #Traning, Testing, Evaluation, Ranking

best_parameters = grid_search7.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search7.best_score_
print("Best result: ", best_result)

# Modified mean square error
r,c=X_.shape
modified_r2 = 1-(1-best_result)*((cv-1)/cv*r-1)/((cv-1)/cv*r-c-1)
print("modified_r2: ", modified_r2)
best_model1 = grid_search7.best_estimator_

Best parameters:  {'C': 1000000, 'epsilon': 10000, 'kernel': 'rbf'}
Best result:  0.7905422233908717
modified_r2:  0.788636256892817


## Model's performance on price prediction for new data

In [13]:
import joblib
print(best_model)
joblib.dump(best_model,"model.pkl")


RandomForestRegressor(max_features='sqrt', n_estimators=450, random_state=1)


['model.pkl']

In [14]:
My_model = joblib.load("model.pkl")
x=[[2,3,4,5,6,7,8,9,1,0,2,3]]
predict=My_model.predict(x)[0]
predict=round(My_model.predict(x)[0],2)
print(predict)

169046.79


In [15]:
price=My_model.predict(X_[:5])
price
DataFrame(price)

,0
0,110257.456889
1,57890.880741
2,89562.659892
3,25661.724265
4,23778.262204
